# AI Mathematical Olympiad - Progress Prize 3 Solution\n\n**Complete MathOlympiad AGI Solution with Symbolic Mathematics**\n\nThis notebook implements:\n- AGI Training System with specialized engine breeding\n- Symbolic Math Solving with SymPy\n- Hyperparameter Optimization\n- Chain-of-Thought Reasoning\n\n**Target:** Maximum accuracy on olympiad-level problems

## Setup

In [ ]:
import os
import sys
import logging

# Setup paths
AIMO3_PATH = '/kaggle/working/kaggle-comps/AI Mathematical Olympiad - Progress Prize 3'
sys.path.insert(0, AIMO3_PATH)
sys.path.insert(0, os.path.join(AIMO3_PATH, 'Data'))

print(f'Path: {AIMO3_PATH}')

In [ ]:
# Install SymPy
!pip install -q sympy

## Import Components

In [ ]:
from aimo3_solution.config import Config
from aimo3_solution.math_solver import MathSolver
from aimo3_solution.utils import setup_logging
from aimo3_solution.engine_breeder import EngineBreeder

try:
    from aimo3_solution.math_olympiad_agi import MathOlympiadAGI
    MATH_AGI_AVAILABLE = True
except ImportError:
    MATH_AGI_AVAILABLE = False

logger = setup_logging(level=logging.INFO)
config = Config()
print('Components loaded')

## Load Engine Configuration

In [ ]:
# Load trained AGI or create default
if MATH_AGI_AVAILABLE and os.path.exists(os.path.join(config.trained_engines_dir, 'math_olympiad_agi.json')):
    math_agi = MathOlympiadAGI(config)
    math_agi.load(config.trained_engines_dir)
    engine_config = math_agi.get_best_engine_config()
    print(f'Loaded trained AGI: {engine_config["name"]}')
else:
    breeder = EngineBreeder(config)
    top = breeder.select_top_engines(1)
    engine_config = breeder.create_engine_config(top[0])
    print(f'Using default engine: {engine_config["name"]}')

## Initialize Solver

In [ ]:
solver = MathSolver(config)
print(f'Solver ready with engine: {engine_config["name"]}')

## Solve Kaggle Problems

In [ ]:
try:
    from kaggle_evaluation.aimo_3_inference_server import AIMO3InferenceServer
    KAGGLE_API = True
except ImportError:
    KAGGLE_API = False

if KAGGLE_API:
    def predict(id: str, problem: str) -> int:
        """Predict function called for each problem"""
        answer, _ = solver.solve(problem, engine_config)
        return int(answer)
    
    # Pass predict function to server
    inference_server = AIMO3InferenceServer(predict)
    
    # Start serving
    count = 0
    for batch in inference_server.serve():
        count += 1
        pid = batch['id'].item()
        problem = batch['problem'].item()
        
        print(f'Solving problem {pid}...')
        
        if count % 10 == 0:
            print(f'Solved {count} problems')
    
    print(f'Completed: {count} problems')
else:
    print('Kaggle API not available')
